In [ ]:
from fastai import *          # Quick access to most common functionality
from fastai.tabular import *  # Quick access to tabular functionality

# Tabular example

Tabular data should be in a Pandas `DataFrame`.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
train_df, valid_df = df[:-2000].copy(),df[-2000:].copy()
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


Convert your `DataFrame` in to a `DataBunch` suitable for modeling by calling `tabular_data_from_df`.

In [ ]:
dep_var = '>=50k'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
data = TabularDataBunch.from_df(path, train_df, valid_df, dep_var, 
                                tfms=[FillMissing, Categorify], cat_names=cat_names)

TypeError: unhashable type: 'numpy.ndarray'

Now you can create a `Learner` with `gen_tabular_dta`, and fit your model

In [ ]:
%debug

> /home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py(1840)_get_item_cache()
   1838         """Return the cached item, item represents a label indexer."""
   1839         cache = self._item_cache
-> 1840         res = cache.get(item)
   1841         if res is None:
   1842             values = self._data.get(item)

ipdb> u
> /home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py(2146)_getitem_column()
   2144         # get column
   2145         if self.columns.is_unique:
-> 2146             return self._get_item_cache(key)
   2147 
   2148         # duplicate columns & possible reduce dimensionality

ipdb> u
> /home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py(2139)__getitem__()
   2137             return self._getitem_multilevel(key)
   2138         else:
-> 2139             return self._getitem_column(key)
   2140 
   2141     def _getitem_column(self, key):

ipdb> u
> /home/ubuntu/fastai/fastai/tabular/data.py(27)__init__

In [ ]:
def show_batch(data, rows:int=10):
    

In [ ]:
learn = get_tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit(1, 1e-2)

Total time: 00:03
epoch  train_loss  valid_loss  accuracy
1      0.328405    0.323188    0.852000  (00:03)

